# TPP Machine Learning Models

Machine learning approaches for predicting and detecting Thyrotoxic Periodic Paralysis episodes.

**Contents:**
1. Feature engineering
2. Classification models (episode vs normal)
3. Regression models (heart rate prediction)
4. Time-series forecasting
5. Model evaluation and comparison
6. Real-time prediction pipeline

## 1. Setup and Data Loading

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso
from sklearn.svm import SVC, SVR
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Configure plotting
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ Libraries loaded successfully")

In [ ]:
# Load sample data
import json

with open('../sample-data/heart-rate-sample.json', 'r') as f:
    data_json = json.load(f)
df_json = pd.DataFrame(data_json)

with open('../sample-data/night-monitoring.json', 'r') as f:
    night_data = json.load(f)
df_night = pd.DataFrame(night_data)

df_csv = pd.read_csv('../sample-data/heart-rate-sample.csv')

# Combine all data
df = pd.concat([df_csv, df_json, df_night], ignore_index=True)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.sort_values('timestamp').reset_index(drop=True)

print(f"Loaded {len(df)} total records")
df.head()

## 2. Feature Engineering

Create additional features from raw data to improve model performance.

In [ ]:
# Time-based features
df['hour'] = df['timestamp'].dt.hour
df['day_of_week'] = df['timestamp'].dt.dayofweek
df['is_night'] = ((df['hour'] >= 22) | (df['hour'] <= 6)).astype(int)
df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)

# Rolling statistics (window=3 for small dataset)
df['hr_rolling_mean'] = df['heartRate'].rolling(window=3, min_periods=1).mean()
df['hr_rolling_std'] = df['heartRate'].rolling(window=3, min_periods=1).std().fillna(0)
df['hrv_rolling_mean'] = df['hrv'].rolling(window=3, min_periods=1).mean()
df['hrv_rolling_std'] = df['hrv'].rolling(window=3, min_periods=1).std().fillna(0)

# Rate of change
df['hr_change'] = df['heartRate'].diff().fillna(0)
df['hrv_change'] = df['hrv'].diff().fillna(0)
df['hr_change_abs'] = df['hr_change'].abs()

# HR/HRV ratio
df['hr_hrv_ratio'] = df['heartRate'] / (df['hrv'] + 1)  # +1 to avoid division by zero

# Encode activity as numeric
le = LabelEncoder()
df['activity_encoded'] = le.fit_transform(df['activity'])

# Create binary target: low HR (potential episode indicator)
# Typically TPP episodes show low heart rate
df['low_hr_episode'] = (df['heartRate'] < 60).astype(int)

print("✓ Feature engineering complete")
print(f"Total features: {len(df.columns)}")
print(f"\nNew columns added: {[col for col in df.columns if col not in ['timestamp', 'heartRate', 'hrv', 'activity', 'device']]}")

## 3. Classification: Episode Detection

Build classification models to detect potential TPP episodes based on low heart rate patterns.

In [ ]:
# Prepare features for classification
feature_cols = ['hrv', 'hour', 'day_of_week', 'is_night', 'is_weekend',
                'hr_rolling_mean', 'hr_rolling_std', 'hrv_rolling_mean', 'hrv_rolling_std',
                'hr_change', 'hrv_change', 'hr_change_abs', 'hr_hrv_ratio', 'activity_encoded']

X = df[feature_cols]
y = df['low_hr_episode']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Training set: {len(X_train)} samples")
print(f"Test set: {len(X_test)} samples")
print(f"Class distribution (train): {y_train.value_counts().to_dict()}")
print(f"Class distribution (test): {y_test.value_counts().to_dict()}")

In [ ]:
# Train multiple classification models
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'SVM': SVC(kernel='rbf', probability=True, random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'KNN': KNeighborsClassifier(n_neighbors=5)
}

results = {}

for name, model in models.items():
    # Train model
    model.fit(X_train_scaled, y_train)
    
    # Predict
    y_pred = model.predict(X_test_scaled)
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1] if hasattr(model, 'predict_proba') else None
    
    # Calculate metrics
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, zero_division=0)
    recall = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    
    results[name] = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'model': model,
        'predictions': y_pred,
        'probabilities': y_pred_proba
    }
    
    print(f"{name}:")
    print(f"  Accuracy:  {accuracy:.3f}")
    print(f"  Precision: {precision:.3f}")
    print(f"  Recall:    {recall:.3f}")
    print(f"  F1 Score:  {f1:.3f}")
    print()

In [ ]:
# Visualize model comparison
results_df = pd.DataFrame({
    'Model': list(results.keys()),
    'Accuracy': [results[m]['accuracy'] for m in results.keys()],
    'Precision': [results[m]['precision'] for m in results.keys()],
    'Recall': [results[m]['recall'] for m in results.keys()],
    'F1 Score': [results[m]['f1'] for m in results.keys()]
})

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Bar plot of all metrics
results_df.set_index('Model')[['Accuracy', 'Precision', 'Recall', 'F1 Score']].plot(
    kind='bar', ax=axes[0], rot=45
)
axes[0].set_title('Model Performance Comparison', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Score')
axes[0].set_ylim(0, 1.0)
axes[0].legend(loc='lower right')
axes[0].grid(True, alpha=0.3)

# Best model confusion matrix
best_model_name = results_df.loc[results_df['F1 Score'].idxmax(), 'Model']
best_predictions = results[best_model_name]['predictions']

cm = confusion_matrix(y_test, best_predictions)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[1])
axes[1].set_title(f'Confusion Matrix: {best_model_name}', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Predicted')
axes[1].set_ylabel('Actual')

plt.tight_layout()
plt.show()

print(f"\n🏆 Best model: {best_model_name} (F1 Score: {results[best_model_name]['f1']:.3f})")

## 4. Regression: Heart Rate Prediction

Build regression models to predict heart rate values.

In [ ]:
# Prepare features for regression (predicting heart rate)
regression_features = ['hrv', 'hour', 'day_of_week', 'is_night', 'is_weekend',
                       'hrv_rolling_mean', 'hrv_rolling_std', 'hrv_change', 'activity_encoded']

X_reg = df[regression_features]
y_reg = df['heartRate']

# Split data
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.3, random_state=42
)

# Scale features
scaler_reg = StandardScaler()
X_train_reg_scaled = scaler_reg.fit_transform(X_train_reg)
X_test_reg_scaled = scaler_reg.transform(X_test_reg)

# Train multiple regression models
regression_models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1.0),
    'Lasso Regression': Lasso(alpha=1.0),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'SVR': SVR(kernel='rbf')
}

regression_results = {}

for name, model in regression_models.items():
    # Train model
    model.fit(X_train_reg_scaled, y_train_reg)
    
    # Predict
    y_pred_reg = model.predict(X_test_reg_scaled)
    
    # Calculate metrics
    mse = mean_squared_error(y_test_reg, y_pred_reg)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test_reg, y_pred_reg)
    r2 = r2_score(y_test_reg, y_pred_reg)
    
    regression_results[name] = {
        'mse': mse,
        'rmse': rmse,
        'mae': mae,
        'r2': r2,
        'predictions': y_pred_reg
    }
    
    print(f"{name}:")
    print(f"  RMSE: {rmse:.2f} bpm")
    print(f"  MAE:  {mae:.2f} bpm")
    print(f"  R²:   {r2:.3f}")
    print()

In [ ]:
# Visualize regression results
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Model comparison
reg_comparison = pd.DataFrame({
    'Model': list(regression_results.keys()),
    'RMSE': [regression_results[m]['rmse'] for m in regression_results.keys()],
    'MAE': [regression_results[m]['mae'] for m in regression_results.keys()],
    'R²': [regression_results[m]['r2'] for m in regression_results.keys()]
})

reg_comparison.set_index('Model')[['RMSE', 'MAE']].plot(kind='bar', ax=axes[0], rot=45)
axes[0].set_title('Regression Model Comparison (Error Metrics)', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Error (bpm)')
axes[0].grid(True, alpha=0.3)

# Best model predictions vs actual
best_reg_model = reg_comparison.loc[reg_comparison['RMSE'].idxmin(), 'Model']
best_reg_preds = regression_results[best_reg_model]['predictions']

axes[1].scatter(y_test_reg, best_reg_preds, alpha=0.6, s=50)
axes[1].plot([y_test_reg.min(), y_test_reg.max()], [y_test_reg.min(), y_test_reg.max()], 
             'r--', lw=2, label='Perfect Prediction')
axes[1].set_xlabel('Actual Heart Rate (bpm)', fontsize=12)
axes[1].set_ylabel('Predicted Heart Rate (bpm)', fontsize=12)
axes[1].set_title(f'Predictions vs Actual: {best_reg_model}', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n🏆 Best regression model: {best_reg_model} (RMSE: {regression_results[best_reg_model]['rmse']:.2f} bpm)")

## 5. Feature Importance Analysis

Identify which features are most important for predictions.

In [ ]:
# Get feature importance from Random Forest classifier
rf_model = results['Random Forest']['model']
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

# Visualize
plt.figure(figsize=(10, 6))
plt.barh(range(len(feature_importance)), feature_importance['importance'])
plt.yticks(range(len(feature_importance)), feature_importance['feature'])
plt.xlabel('Importance Score', fontsize=12)
plt.title('Feature Importance for Episode Detection', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

print("\nTop 5 Most Important Features:")
print(feature_importance.head())